In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
from gausspyplus.decompose import GaussPyDecompose
from gausspyplus.finalize import Finalize
from gausspyplus.plotting import plot_spectra
from gausspyplus.prepare import GaussPyPrepare
from gausspyplus.spatial_fitting import SpatialFitting
from gausspyplus.training import GaussPyTraining
from gausspyplus.training_set import GaussPyTrainingSet

In [2]:
def run(path,filename,gpydir):
    
    gpydir=gpydir
    filepath=path+filename
    basename=os.path.splitext(filename)[0]
    pickledir=basename+'pickle'

    '''
    training set
    '''

    
    training = GaussPyTrainingSet(config_file='gausspy+.ini')
    training.path_to_file = filepath 
    training.dirpath_gpy = gpydir
    training.n_spectra = 100
    training.random_seed = 111
    training.rchi2_limit = 1.2
    training.max_fwhm = 50.
    training.filename_out = basename+'-training_set_{}_spectra.pickle'.format(training.n_spectra)
    training.decompose_spectra()  

    '''
    train
    '''

    
    train = GaussPyTraining(config_file='gausspy+.ini')
    train.dirpath_gpy = gpydir
    train.path_to_training_set = os.path.join(train.dirpath_gpy, 'gpy_training', basename+'-training_set_100_spectra.pickle')
    train.two_phase_decomposition = True
    train.alpha1_initial = 5.
    train.alpha2_initial = 7.
    train.training()

    '''
    prepare
    '''

    
    prepare = GaussPyPrepare(config_file='gausspy+.ini')
    prepare.path_to_file = filepath
    prepare.dirpath_gpy = gpydir
    prepare.prepare_cube()
    prepare.produce_noise_map()

    '''
    decompose
    '''
    
    decompose = GaussPyDecompose(config_file='gausspy+.ini')
    decompose.path_to_pickle_file = os.path.join(gpydir, 'gpy_prepared', basename+'.pickle')
    decompose.alpha1 = train.alpha1
    decompose.alpha2 = train.alpha2
    decompose.suffix = '_g+'
    decompose.decompose()

    '''
    refit 1
    '''

    sp = SpatialFitting(config_file='gausspy+.ini')
    sp.path_to_pickle_file = os.path.join(gpydir, 'gpy_prepared', basename+'.pickle')
    sp.path_to_decomp_file = os.path.join(gpydir, 'gpy_decomposed', basename+'_g+_fit_fin.pickle')
    sp.refit_blended = False
    sp.refit_neg_res_peak = True
    sp.refit_broad = False
    sp.flag_residual = True
    sp.refit_residual = False
    sp.refit_ncomps = False
    sp.max_diff_comps = 1
    sp.max_jump_comps = 2
    sp.n_max_jump_comps = 1
    sp.mean_separation = 2.
    sp.fwhm_separation = 4.
    sp.spatial_fitting()

    # '''
    # refit 2
    # '''

    # sp = SpatialFitting(config_file='gausspy+.ini')
    # sp.path_to_pickle_file = os.path.join(
    #     gpydir, 'gpy_prepared', basename+'.pickle')
    # sp.path_to_decomp_file = os.path.join(
    #     gpydir, 'gpy_decomposed',
    #     basename+'_g+_fit_fin_sf-p1.pickle')
    # sp.refit_blended = True
    # sp.refit_neg_res_peak = True
    # sp.refit_broad = True
    # sp.flag_residual = True
    # sp.refit_residual = False
    # sp.refit_ncomps = True
    # sp.max_diff_comps = 1
    # sp.max_jump_comps = 2
    # sp.n_max_jump_comps = 1
    # sp.mean_separation = 4.
    # sp.fwhm_separation = 4.
    # sp.min_weight = 0.6
    # sp.spatial_fitting(continuity=True)


In [3]:
path ='/home/shuoshuo/hi4pi_subcubes/'
gpydir = 'decomp'

In [4]:
for i in range(1, 2):
    for j in range(2):
        filename='subcube_'+str(i)+'_'+str(j)+'.fits'
        run(path, filename, gpydir)

decompose 100 spectra ...
using 12 out of 16 cpus


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  8.43it/s]

SUCCESS


SAVED FILE: 'subcube_1_0-training_set_100_spectra.pickle' in 'decomp/gpy_training'

GaussPy training
Using training set: decomp/gpy_training/subcube_1_0-training_set_100_spectra.pickle
Training...



5.0, 0.9, 0.01929550965726823, 0.0
iter 0: F1=71.9%, alpha=[5.0, 7.0], p=[0.00, 0.00]  (Convergence testing begins in 20 iterations)

4.982634041308459, 0.9, 0.012461099501272055, -0.013892766953232894
iter 1: F1=71.8%, alpha=[4.98, 7.0], p=[-0.01, 0.00]  (Convergence testing begins in 19 iterations)

4.957526284804081, 0.9, 0.003512538711430091, -0.020086205203502062
iter 2: F1=71.8%, alpha=[4.96, 7.0], p=[-0.02, 0.00]  (Convergence testing begins in 18 iterations)

4.934278794760292, 0.9, 0.00800178884673386, -0.018597992035031298
iter 3: F1=71.7%, alpha=[4.93, 7.0], p=[-0.02, 0.00]  (Convergence testing begins in 17 iterations)

4.9084791927632, 0.9, -0.022197786136097797, -0.020639681597673844
iter 4: F1=71.7%, alpha=[4.91, 7.0], p=[-0.02, 0.00]  (Convergence testing begins in 16 iterations)

4.907817518688015, 0.9, -0.017777894834491992, -0.0005293392601480207
iter 5: F1=71.7%, alpha=[4.91, 7.0], p=[-0.00, 0.00]  (Convergence testing begins in 15 iterations)

4.923288284778909, 0

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3418.66it/s]


...

546446it [00:01, 350832.18it/s]



pickle dump dictionary...
For GaussPyDecompose: 'path_to_pickle_file' = 'decomp/gpy_prepared/subcube_1_0.pickle'

SAVED FILE: 'subcube_1_0_noise_map.fits' in 'decomp/gpy_maps'

pickle load 'subcube_1_0.pickle'...

GaussPy decomposition

decomposition settings:
GaussPy:
Two phase decomposition: True
alpha1: 4.947959745921423
alpha2: 7.0
SNR1: 3.0
SNR2: 3.0

improve_fitting: True
min_fwhm: 1.0
max_fwhm: None
snr: 3.0
snr_fit: 1.5
significance: 5.0
snr_negative: 3.0
rchi2_limit: None
max_amp_factor: 1.1
neg_res_peak: True
broad: True
blended: True
fwhm_factor: 2.0
separation_factor: 0.8493218
exclude_means_outside_channel_range: True
min_pvalue: 0.01
max_ncomps: None

decomposing data...
using 12 out of 16 cpus


100%|██████████████████████████████████████████████████████████████████████████████| 546k/546k [3:15:24<00:00, 46.6it/s]
546445it [00:26, 20692.53it/s] 



pickle dump GaussPy final results...
SAVED FILE: 'subcube_1_0_g+_fit_fin.pickle' in 'decomp/gpy_decomposed'

Spatial refitting - Phase 1

Flagging:
 - Blended components: False
 - Negative residual features: True
 - Broad components: False
   flagged if FWHM of broadest component in spectrum is:
   >= 2.0 times the FWHM of second broadest component
   or
   >= 2.0 times any FWHM in >= 50% of its neigbors
 - High reduced chi2 values (> None): False
 - Non-Gaussian distributed residuals: True
 - Differing number of components: False

For phase 1:
Exclude flagged spectra as possible refit solutions in first refit attempts: False
Use also flagged spectra as refit solutions in case no new best fit could be obtained from unflagged spectra: False

Refitting:
 - Blended components: False
 - Negative residual features: True
 - Broad components: False
   try to refit if FWHM of broadest component in spectrum is:
   >= 2.0 times the FWHM of second broadest component
   or
   >= 2.0 times any FWH

ValueError: not enough values to unpack (expected 4, got 3)